In [ ]:
!pip install transformers
!pip install accelerate
!pip install stop-words
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re
import string
import swifter
import string
import csv
from stop_words import get_stop_words
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
# Download necessary NLTK data
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

file_path = 'translated_output_complete.csv'
print("Loading data...")
try:
    df = pd.read_csv(file_path, encoding='utf-8', nrows=200000)
    print("Data loaded successfully with UTF-8 encoding.")
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, encoding='latin1', nrows=200000)
        print("Data loaded successfully with Latin-1 encoding.")
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='iso-8859-1', nrows=200000)
        print("Data loaded successfully with ISO-8859-1 encoding.")

def clean_text(text, lang):
    if not isinstance(text, str):
        return ""
    text = re.sub(r"http\S+|www\S+|https\S+|\@\w+|\#", '', text, flags=re.MULTILINE)
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()

    words = text.split()

    if lang == 'hebrew':
        stop_words = set(stopwords.words('hebrew'))
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words if word not in stop_words]
        
    else:
        stop_words = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return ' '.join(words)

In [ ]:
print("Applying text preprocessing...")
# df['original_text'] = df.apply(lambda row: clean_text(row['original_text'], 'english'), axis=1)
df['text'] = df.apply(lambda row: clean_text(row['text'], 'english'), axis=1)
df['translated_data'] = df.apply(lambda row: clean_text(row['translated_data'], 'hebrew'), axis=1)
# df['translated_text'] = df.apply(lambda row: clean_text(row['translated_text'], 'hebrew'), axis=1)
print("Text preprocessing completed.")


# Prepare data
print("Preparing data for training...")
# english_texts = df['original_text'].values
# hebrew_texts = df['translated_text'].values
english_texts = df['text'].values
hebrew_texts = df['translated_data'].values
classifications = np.where(df['labels'] == 'suicide', 1, 0)

combined_texts = np.concatenate((english_texts, hebrew_texts))
combined_labels = np.concatenate((classifications, classifications))
print("Data preparation completed.")

# Split data
print("Splitting data into training, validation, and test sets...")
train_texts, test_texts, train_labels, test_labels = train_test_split(combined_texts, combined_labels, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)
print("Data splitting completed.")

# Tokenization and Encoding
print("Starting tokenization and encoding...")
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [ ]:
def encode_data(texts, max_length=256):
    print(f"Encoding {len(texts)} texts...")
    encoded_data = tokenizer(text=texts.tolist(), padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    print("Encoding completed.")
    return encoded_data

print("Starting data encoding...")

# Encode training data
train_encoded = encode_data(train_texts)
print("Training data encoded.")

# Encode validation data
val_encoded = encode_data(val_texts)
print("Validation data encoded.")

# Encode test data
test_encoded = encode_data(test_texts)
print("Test data encoded.")

print("Creating DataLoaders...")

# Create DataLoaders
train_dataset = TensorDataset(train_encoded['input_ids'], train_encoded['attention_mask'], torch.tensor(train_labels))
val_dataset = TensorDataset(val_encoded['input_ids'], val_encoded['attention_mask'], torch.tensor(val_labels))
test_dataset = TensorDataset(test_encoded['input_ids'], test_encoded['attention_mask'], torch.tensor(test_labels))

batch_size = 32
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
print("Training DataLoader created.")

val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
print("Validation DataLoader created.")  # Fixed the missing quotation mark

test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
print("Test DataLoader created.")

print("All DataLoaders are ready.")



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

class EnhancedSuicideClassifier(nn.Module):
    def __init__(self, dropout_rate=0.4):
        super(EnhancedSuicideClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-uncased')

        # Freeze BERT layers
        for param in self.bert.parameters():
            param.requires_grad = False

        # Multi-head self-attention
        self.multihead_attn = nn.MultiheadAttention(embed_dim=768, num_heads=8)

        # Bi-directional LSTM
        self.bilstm = nn.LSTM(768, 512, bidirectional=True, batch_first=True)

        # Fully connected layers
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6 = nn.Linear(128, 64)
        self.fc7 = nn.Linear(64, 1)

        # Batch normalization layers
        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(512)
        self.bn3 = nn.BatchNorm1d(512)
        self.bn4 = nn.BatchNorm1d(256)
        self.bn5 = nn.BatchNorm1d(128)
        self.bn6 = nn.BatchNorm1d(64)

        # Dropout
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state

        # Multi-head self-attention
        attn_output, _ = self.multihead_attn(sequence_output.transpose(0, 1), sequence_output.transpose(0, 1), sequence_output.transpose(0, 1))
        attn_output = attn_output.transpose(0, 1)

        # Bi-directional LSTM
        lstm_output, _ = self.bilstm(attn_output)
        lstm_output = lstm_output[:, -1, :]  # Take the last hidden state

        # Forward pass through fully connected layers with batch normalization
        x = self.fc1(lstm_output)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.bn3(x)
        x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc4(x)
        x = self.bn4(x)
        x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc5(x)
        x = self.bn5(x)
        x = torch.relu(x)
        x = self.dropout(x)

        x = self.fc6(x)
        x = self.bn6(x)
        x = torch.relu(x)
        x = self.dropout(x)

        output = self.fc7(x)

        return output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EnhancedSuicideClassifier(dropout_rate=0.5).to(device)

optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)

criterion_main = nn.BCEWithLogitsLoss()

epochs = 30
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


def train_epoch(model, dataloader, optimizer, scheduler, criterion_main):
    model.train()
    total_loss = 0
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader), desc="Training")

    for batch_idx, batch in progress_bar:
        input_ids, attention_mask, labels = [b.to(device) for b in batch]
        optimizer.zero_grad()
        main_output = model(input_ids, attention_mask)
        loss = criterion_main(main_output.squeeze(), labels.float())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()

        progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

    return total_loss / len(dataloader)

def evaluate(model, dataloader):
    model.eval()
    main_predictions = []
    true_labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids, attention_mask, labels = [b.to(device) for b in batch]
            main_output = model(input_ids, attention_mask)
            main_preds = torch.sigmoid(main_output.squeeze()) > 0.5
            main_predictions.extend(main_preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    return (accuracy_score(true_labels, main_predictions),
            precision_score(true_labels, main_predictions),
            recall_score(true_labels, main_predictions),
            f1_score(true_labels, main_predictions))

start_epoch = 0
best_val_f1 = 0
no_improve = 0


patience = 7
for epoch in range(start_epoch, epochs):
    train_loss = train_epoch(model, train_dataloader, optimizer, scheduler, criterion_main)
    val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_dataloader)

    print(f"\nEpoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Accuracy: {val_acc:.4f}, Precision: {val_prec:.4f}, Recall: {val_rec:.4f}, F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        no_improve = 0
        # print("New best model saved!")
    else:
        no_improve += 1

    if no_improve == patience:
        print("Early stopping")
        break

    print() 
# model.load_state_dict(torch.load(checkpoint_path)['model_state_dict'])

test_acc, test_prec, test_rec, test_f1 = evaluate(model, test_dataloader)
print("Test Results:")
print(f"Accuracy: {test_acc:.4f}, Precision: {test_prec:.4f}, Recall: {test_rec:.4f}, F1: {test_f1:.4f}")



In [ ]:
# def test_model(model, test_loader, criterion):
#     model.eval()
#     total_test_loss = 0.0
#     all_predictions = []
#     all_labels = []

#     with torch.no_grad():
#         for batch in test_loader:
#             input_ids, attention_mask, labels = batch
#             input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

#             outputs = model(input_ids, attention_mask)
#             loss = criterion(outputs.squeeze(), labels.float())

#             total_test_loss += loss.item()

#             predictions = torch.sigmoid(outputs).cpu().numpy()
#             predictions = np.where(predictions > 0.5, 1, 0)
#             all_predictions.extend(predictions.flatten())
#             all_labels.extend(labels.cpu().numpy().flatten())

#     avg_test_loss = total_test_loss / len(test_loader)
#     print(f'Test Loss: {avg_test_loss:.4f}')

#     accuracy = accuracy_score(all_labels, all_predictions)
#     precision = precision_score(all_labels, all_predictions)
#     recall = recall_score(all_labels, all_predictions)
#     f1 = f1_score(all_labels, all_predictions)
#     auc = roc_auc_score(all_labels, all_predictions)
#     conf_matrix = confusion_matrix(all_labels, all_predictions)

#     print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}, AUC: {auc:.4f}')
#     print("Confusion Matrix:")
#     print(conf_matrix)

# # Load the best model and evaluate on the test set
# model.load_state_dict(torch.load('200k_Remove_stopWords.pth'))
# criterion = nn.BCEWithLogitsLoss()  # Initialize the criterion without any arguments

# # Test the model
# test_model(model, test_dataloader, criterion)


# # # Load the best model and evaluate on the test set
# # model.load_state_dict(torch.load('200k_Remove_stopWords.pth'))
# # test_model(model, test_dataloader, nn.BCEWithLogitsLoss('200k_Remove_stopWords.pth'))

# # # Function to predict custom strings
# # def predict_custom_string(model, tokenizer, text, lang='english'):
# #     cleaned_text = clean_text(text, lang)
# #     # print(cleaned_text)
# #     encoded_input = tokenizer(text, padding=True, truncation=True, max_length=256, return_tensors='pt')

# #     input_ids = encoded_input['input_ids'].to(device)
# #     attention_mask = encoded_input['attention_mask'].to(device)

# #     model.eval()
# #     with torch.no_grad():
# #         output = model(input_ids, attention_mask)

# #     prediction = torch.sigmoid(output).cpu().numpy()
# #     return prediction[0][0]

# # # Custom Predictions
# # custom_texts = [
# #     "אשתי עזבה החיים בזבל לא בא לי לחיות",
# #     "אני רוצה להתאבד",
# #     "i wanna live forever",
# #     "אני אוהב את החיים האלה",
# #     "אני שונא את האדם הזה",
# #     "אני לא יכול לשאת את זה יותר אני אובדני",
# #     "התאבדות",
# #     "אושר והנאה ושמחה מהחיים",
# #     "want kill myself",
# #     "really good",
# #     "שמחה ואושר",
# #     "i love you",
# #     "i hate you and i dont like my life",
# #     "i want to commit suicide",
# #     "אני רוצה לבתאבד אבל אני אוהב את המשפחה שלי אז אני בוודאות לא אתאבד",
# #     "איזה כיף יש מבצע בזארה",
# #     "i am tired of losing in call of duty",
# #     "בזמן האחרון אני קצת מוטרד מהמצב וחושב עלייך די הרבה השיחות שלנו כמו שירים בהופעות אני זוכר את הכל בעל פה",
# #     "זה אני האחד שסגד ליופיך שנתן את הכל בשבילך ששברת את ליבו בלכתך, זה אני שהיית ונשארת בדמי"
# # ]

# # for text in custom_texts:
# #     lang = 'hebrew' if re.search(r'[א-ת]', text) else 'english'
# #     prediction = predict_custom_string(model, tokenizer, text, lang)
# #     print(f'Prediction for "{text}": {prediction:.4f}')